In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('Classificacao com Spark')\
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [4]:
spark

In [5]:
dataset = spark.read.parquet('Dataset Modelo/', inferSchema = True)

In [6]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Conjuge: integer (nullable = true)
 |-- Dependentes: integer (nullable = true)
 |-- TelefoneFixo: integer (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = true)
 |-- SegurancaOnline: integer (nullable = true)
 |-- BackupOnline: integer (nullable = true)
 |-- SeguroDispositivo: integer (nullable = true)
 |-- SuporteTecnico: integer (nullable = true)
 |-- TVaCabo: integer (nullable = true)
 |-- StreamingFilmes: integer (nullable = true)
 |-- ContaCorreio: integer (nullable = true)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContrato_DoisAnos

In [7]:
dataset.toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,9465,0,60,103.614223,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
2,2122,0,12,75.850000,0,0,0,1,1,0,...,1,0,0,0,1,0,0,0,0,1
3,3997,0,69,61.450000,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
4,6654,0,7,86.500000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,1476,0,2,20.250000,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,1
10344,1138,0,11,65.150000,0,1,1,1,1,1,...,1,0,0,1,0,0,1,0,0,0
10345,5125,0,11,94.000000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
10346,2376,0,11,61.250000,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


# Regressão Logistica

##### A regressão logística é um método popular para prever uma resposta categórica, podemos prever um resultado binário usando regressão logiística binominal ou podemos prever um resultado multiclasse usando regressão logistica multinominal

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
# Renomeando a coluna
dataset = dataset.withColumnRenamed('Churn', 'label')

In [10]:
dataset.limit(1).toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,label,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [11]:
# Precisamos criar as features, que são todas as colunas que serão usadas na previsão
# Importante: O 'label' não pode estar na features, pois é o valor que precisamos descobrir
X = dataset.columns
X.remove('label')
X.remove('id')

In [12]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [13]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [14]:
# No transform passamos o dataset que é usada para previsão, e faremos o select com as variaveis que criamos
dataset_preparado = assembler.transform(dataset).select('features', 'label')

In [15]:
dataset_preparado.show(10, False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

# Ajuste e previsão

#### Para começar o treinamento precisamos separa nossos dados entre "treino" e "teste"

In [16]:
v_seed = 101

In [17]:
# Por padrão são usados 70% para treino e 30% para teste
treino, teste = dataset_preparado.randomSplit([0.7, 0.3], seed=v_seed)

In [18]:
treino.count()

7206

In [19]:
teste.count()

3142

# Usando a Regressão Logística

In [20]:
from pyspark.ml.classification import LogisticRegression

In [21]:
lr = LogisticRegression()

In [22]:
# Criando Modelo para ajustar o modelo de treino
modelo_lr = lr.fit(treino)

In [23]:
# Classificando o modelo de teste
previsoes_lr_teste = modelo_lr.transform(teste)

In [24]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,

# Métricas

In [25]:
resumo_lr_treino = modelo_lr.summary

In [26]:
resumo_lr_treino.accuracy

0.7849014709963918

In [27]:
print("Acurácia: %f" % resumo_lr_treino.accuracy)
print("Precisão: %f" % resumo_lr_treino.precisionByLabel[1])
print("Recall: %f" % resumo_lr_treino.recallByLabel[1])
print("F1: %f" % resumo_lr_treino.fMeasureByLabel()[1])

Acurácia: 0.784901
Precisão: 0.770686
Recall: 0.812517
F1: 0.791049


In [28]:
from pyspark.sql import functions as f

In [29]:
# Precisamos criar uma matriz de confusão, para ver como os dados de teste se performou
previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()

1256

In [30]:
tp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
tn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
fp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
fn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()
print(tp, tn, fp, fn)

1256 1179 400 307


In [31]:
# Criando uma def para exibição
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn

  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4, 'Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

In [32]:
calcula_mostra_matriz_confusao(previsoes_lr_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


##### Conclusão: Vimos que os valores não foram como esperamos já que esse modelo mostrou valor Não-Churn para valores que deveriam ser e mostrou Churn para valores que não deveriam ser, com isso podemos ver se outro modelo tem acertividade maior

# Árvores de Decisão

In [33]:
from pyspark.ml.classification import DecisionTreeClassifier

In [34]:
dtc = DecisionTreeClassifier(seed=v_seed)

In [35]:
modelo_dtc = dtc.fit(treino)

In [36]:
previsoes_dtc_treino = modelo_dtc.transform(treino)

In [37]:
previsoes_dtc_treino.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,

# Métricas Arvore de Decisão

In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
evaluator = MulticlassClassificationEvaluator()

In [40]:
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})

0.7917013599777962

In [41]:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.791701
Precisão: 0.805090
Recall: 0.770978
F1: 0.787664


In [42]:
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [43]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [62.0,128.0]|[0.32631578947368...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [51.0,141.0]| [0.265625,0.734375]|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [63.0,118.0]|[0.34806629834254...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,

In [44]:
evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'accuracy'})

0.7714831317632082

In [45]:
print('Decision Tree Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2784         827
Real
     Não-Churn    674         2921
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.791701
Precisão: 0.805090
Recall: 0.770978
F1: 0.787664

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.771483
Precisão: 0.778510
Recall: 0.755598
F1: 0.766883


# Random Forest

In [46]:
from pyspark.ml.classification import RandomForestClassifier

In [47]:
rfc = RandomForestClassifier(seed=v_seed)

In [48]:
modelo_rfc = rfc.fit(treino)

In [49]:
previsoes_rfc_treino = modelo_rfc.transform(treino)

In [50]:
previsoes_rfc_treino.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[15.0052773466704...|[0.75026386733352...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[16.9295040273249...|[0.84647520136624...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[8.59288938528764...|[0.42964446926438...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.59647122885698...|[0.27982356144284...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.21616013157118...|[0.26080800657855...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[5.45640255581361...|[0.27282012779068...|       1.0|
|(24,[0,1,2,3,4,

# Métricas Random Forest

In [51]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [52]:
previsoes_lr_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1680594619286...|[0.45808374494006...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.4170949608173...|[0.19511740608882...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.14194260698794...|[0.53542619200881...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67046644011599...|[0.66160759507905...|       0.0|
|(24,[0,1,2,3,4,

In [53]:
print('Random Forest Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2950         661
Real
     Não-Churn    884         2711
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.785595
Precisão: 0.769431
Recall: 0.816948
F1: 0.792478

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.770210
Precisão: 0.751345
Recall: 0.804223
F1: 0.776885


In [54]:
calcula_mostra_matriz_confusao(previsoes_dtc_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243


In [55]:
calcula_mostra_matriz_confusao(previsoes_lr_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


In [56]:
calcula_mostra_matriz_confusao(previsoes_rfc_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163


In [74]:
def comparacao_modelo():
    print('Previsão Teste Regressão Logística: ')
    print("="*50)
    print("="*50)
    calcula_mostra_matriz_confusao(previsoes_lr_teste)
    print()
    print("Acurácia: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "accuracy"}))
    print("Precisão: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
    print("Recall: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
    print("F1: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
    print("="*50)
    print("="*50)
    print('Previsão Teste Decision Tree: ')
    calcula_mostra_matriz_confusao(previsoes_dtc_teste)
    print()
    print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
    print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
    print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
    print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
    print("="*50)   
    print("="*50)
    print('Previsão Teste Random Forest: ')
    calcula_mostra_matriz_confusao(previsoes_rfc_teste)
    print()
    print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
    print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
    print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
    print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
    print("="*50)
    print("="*50)

In [75]:
comparacao_modelo()

Previsão Teste Regressão Logística: 
                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179

Acurácia: 0.774984
Precisão: 0.758454
Recall: 0.803583
F1: 0.780367
Previsão Teste Decision Tree: 
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243

Acurácia: 0.771483
Precisão: 0.778510
Recall: 0.755598
F1: 0.766883
Previsão Teste Random Forest: 
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163

Acurácia: 0.770210
Precisão: 0.751345
Recall: 0.804223
F1: 0.776885


# Técnicas de Otimização

###### Até então, quando treinamos os nossos algoritmos, utilizamos os hiperparâmetros padrões, os quais podem não ser adequados para o problema que precisamos resolver (classificar se um cliente vai ou não cancelar um serviço) e para o nosso conjunto de dados. Cada tipo de dados irá funcionar com hiperparâmetros específicos e devemos investigar isso, em vez de utilizar somente o padrão. Pensando nisso, um dos hiperparâmetros que podemos explorar para essa situação é o maxDepth: ele fala a respeito da profundidade máxima que a árvore pode alcançar. Assim, podemos diminuir o efeito do algoritmo como especialista em dados de treino, criando uma solução mais geral.
###### Cross-Validation: Existe uma técnica por meio da qual podemos variar os grupos de dados, os quais serão ajustados, e entender como o modelo performa na média. De maneira geral, vamos compreender como o algoritmo se comporta com os dados. Trata-se de uma etapa importante para diminuirmos o efeito da aleatoriedade ao selecionar um grupo de dados.

# Arvore de Decisão - Tuning

In [76]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

In [78]:
dtc = DecisionTreeClassifier(seed = v_seed)

In [80]:
grid = ParamGridBuilder()\
    .addGrid(dtc.maxDepth, [2, 5, 10])\
    .addGrid(dtc.maxBins, [10, 32, 45])\
    .build()

In [81]:
evaluator = MulticlassClassificationEvaluator()

In [82]:
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds= 3,
    seed = v_seed
)

In [83]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [85]:
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [86]:
print('Decision Tree Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1327         236
Real
     Não-Churn    423         1156
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.790261
Precisão: 0.758286
Recall: 0.849008
F1: 0.801087


# Random Forest- Tuning

In [89]:
rfc = RandomForestClassifier(seed = v_seed)

In [90]:
grid = ParamGridBuilder()\
    .addGrid(rfc.maxDepth, [2, 5, 10])\
    .addGrid(rfc.maxBins, [10, 32, 45])\
    .addGrid(rfc.numTrees, [10, 20, 50])\
    .build()

In [91]:
evaluator = MulticlassClassificationEvaluator()

In [92]:
rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator= evaluator,
    numFolds= 3,
    seed= v_seed)

In [93]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [94]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [96]:
previsoes_rfc_cv_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[18.8458622358365...|[0.94229311179182...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.00338773070618...|[0.35016938653530...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[11.4626166118451...|[0.57313083059225...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[10.1291848964521...|[0.50645924482260...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[10.7004509316445...|[0.53502254658222...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.14460132501597...|[0.35723006625079...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2.68605623364822...|[0.13430281168241...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.48796840365584...|[0.47439842018279...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[15.0032595338437...|[0.75016297669218...|       0.0|
|(24,[0,1,2,3,4,

In [97]:
print('Random Forest Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1322         241
Real
     Não-Churn    347         1232
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.812858
Precisão: 0.792091
Recall: 0.845809
F1: 0.818069


# Modelo Final

In [98]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [99]:
print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees)

10
45
20


In [100]:
rfc_tunning = RandomForestClassifier(maxDepth= 10, maxBins=45, numTrees=10,seed = v_seed)

In [101]:
modelo_rfc_tunning = rfc_tunning.fit(dataset_preparado) 

In [102]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [103]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

In [109]:
novo_cliente = spark.createDataFrame(novo_cliente)

TypeError: data is already a DataFrame

In [110]:
novo_cliente.toPandas()

,BackupOnline,Conjuge,ContaCorreio,Dependentes,Internet_DSL,Internet_FibraOptica,Internet_Nao,Mais65anos,MaisDeUmaLinhaTelefonica,MesesCobrados,...,MetodoPagamento_DebitoEmConta,SegurancaOnline,SeguroDispositivo,StreamingFilmes,SuporteTecnico,TVaCabo,TelefoneFixo,TipoContrato_DoisAnos,TipoContrato_Mensalmente,TipoContrato_UmAno
0,0,0,1,0,1,0,0,0,0,45.305408,...,0,0,0,1,0,1,0,0,1,0


In [111]:
# Fazer a transformação dos dados do novo cliente
assembler = VectorAssembler(inputCols= X, outputCol= 'features')

In [112]:
novo_cliente_prep = assembler.transform(novo_cliente).select('features')

In [114]:
novo_cliente_prep.show(truncate = False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



In [115]:
modelo_rfc_tunning.transform(novo_cliente_prep).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[1.24640781461703...|[0.12464078146170...|       1.0|
+--------------------+--------------------+--------------------+----------+

